#### Requirements

In [1]:
import urllib
import urllib.request
from urllib.request import urlopen, Request
import urllib.parse
from urllib.error import HTTPError
from urllib.error import URLError
from http.cookiejar import CookieJar
from bs4 import BeautifulSoup
import lxml
import cchardet
import json, os
import pandas as pd
from fpdf import FPDF


#### Request

In [4]:
base_url = "https://api.elsevier.com/content/search/scopus?"

apiKey = "a981e389772e23df3d045cc51e2bcfc6"

query = "TITLE-ABS-KEY ({multiple sclerosis} AND {corpus callosum})"

view = "STANDARD"


try:
    url = base_url + "query=" + urllib.parse.quote(query) + "&apiKey=" + apiKey + "&view=" + view
    fetch_results = json.dumps(json.loads(urlopen(url).read()), indent=4)
    with open("scopus_search_results0.json", "w") as output:
        output.write(fetch_results)
    with open("scopus_search_results0.json", "r") as search_results:
        search_results = json.load(search_results)
    total_results = int(search_results["search-results"]["opensearch:totalResults"])

    start = 26
    int(total_results/(start-1))
    
    total_files = int(total_results/(start-1))
    counter = 1
    while counter <= total_files:
        url = base_url + "query=" + urllib.parse.quote(query) + "&apiKey=" + apiKey + "&view=" + view + "&start=" + str(start)
        fetch_results = json.dumps(json.loads(urlopen(url).read()), indent=4)
        with open(f"scopus_search_results{counter}.json", "w") as output:
            output.write(fetch_results)
        if start+25 < total_results:
            start += 25
            counter += 1
        else:
            break

except HTTPError as e:
    print(e)
except URLError as e:
    print('Not found!')

KeyboardInterrupt: 

#### Merge and save results

In [5]:
total_results = int(search_results["search-results"]["opensearch:totalResults"])

IDs = [None] * total_results
titles = [None] * total_results
links = [None] * total_results

counter = 0
while counter <= total_files:
    with open(f"scopus_search_results{counter}.json", "r") as search_results:
        search_results = json.load(search_results)
    for index, entry in enumerate(search_results["search-results"]["entry"]):
        IDs[index+(counter*25)] = entry["eid"]
        titles[index+(counter*25)] = entry["dc:title"]
        for link in entry["link"]:
            if link["@ref"] == "scopus":
                links[index+(counter*25)] = entry["link"][entry["link"].index(link)]["@href"]
    counter += 1

In [21]:
abstracts3 = [None] * total_results

for index, link in enumerate(links):
    abstracts3[index+261] = BeautifulSoup(urllib.request.build_opener(urllib.request.HTTPCookieProcessor(CookieJar())).open(Request(links[index+261], headers = {'User-Agent': 'Mozilla/5.0'})).read(), 'html.parser').select("section[id = abstractSection] > p")[0].replace_with("</p>", "")
    print(index+261)


261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456


IndexError: list index out of range

In [18]:
articles_list3 = {'IDs': IDs, 'Title': titles, 'Abstract': abstracts3}
articles_list3 = pd.DataFrame(articles_list2)
articles_list3.to_csv("scopus_articles2.csv")

In [19]:
print(articles_list3)

                     IDs                                              Title  \
0     2-s2.0-85127375931  MR volumetry in detection of brain atrophic ch...   
1     2-s2.0-85124238043  Neuroradiological differentiation of white mat...   
2     2-s2.0-85125650520  Ceramide kinase knockout ameliorates multiple ...   
3     2-s2.0-85124555237  Expanding the phenotype of HNRNPU-related neur...   
4     2-s2.0-85116499252  Correlations between hippocampal functional co...   
...                  ...                                                ...   
1508   2-s2.0-0014791756  LIPIDS AND PROTEINS IN MULTIPLE SCLEROSIS WHIT...   
1509   2-s2.0-0014719498  ACID PROTEINASE ACTIVITY OF WHITE MATTER AND P...   
1510   2-s2.0-0004777895  THE RELATIONSHIP BETWEEN ENZYME ACTIVITY AND N...   
1511  2-s2.0-50549161122  Observations on the presence of lysolecithin i...   
1512                None                                               None   

                                               Abst